In [102]:
import numpy as np

class LABS:
    def __init__(self, length = 4, mutation_rate = 0.5, crossover_rate = 0.9, init_pop_size = 10, num_epochs = 50):
        '''Generate initial binary sequence and signed sequence'''
        self.init_pop_size = init_pop_size
        self.length = length
        self.num_epochs = num_epochs
        self.binary_sequence = None 
        self.signed_sequence = None
        self.mutation_rate  = mutation_rate
        self.crossover_rate = crossover_rate
        self.population = []
        self.signed_population = []
        self.child = None
        self.LABS_scores = []

    def generate_population(self):
        '''Generate population and signed population'''
        #print("Init pop size "+ str(self.init_pop_size))
        for i in range(self.init_pop_size):
            self.population.append(self.generate_binary())


        #print("Length of population " + str(len(self.population)))
        for i in range(len(self.population)):
            binary = self.population[i]
            signed = self.generate_signed(binary)
            self.signed_population.append(signed)


    def generate_binary(self):
        return np.random.choice([0, 1], size=self.length)
        #self.population.append(self.binary_sequence)

    def generate_signed(self, binary):
        signed_sequence = binary * 2 - 1
        #print(self.signed_sequence)
        return signed_sequence

    def calculate_rk(self, k):
        n = self.length
        r_k = 0
        if (k < n):
            for i in range( n - k):
                r_k += self.signed_sequence[i] * self.signed_sequence[i+k]
        else:
            print("Index out of range")
        return r_k
    
    def calculate_autocorrelation(self):
        autocorr_score = 0
        n = self.length
        for k in range(1, n):
            autocorr_score += (self.calculate_rk(k))**2
        return autocorr_score

    def calculate_LABS(self):
        ''' Calculate the LABS score or fitness score of each individual in the population'''
        #print("Initial population size: " + str(len(self.population)))
        n = self.length
        self.LABS_scores = []
        #print("Pop size" + str(self.init_pop_size))
        for i in range(self.init_pop_size):
            self.binary_sequence = self.population[i]
            self.signed_sequence = self.signed_population[i]
            LABS_score = n**2/ (2 * self.calculate_autocorrelation())
            self.LABS_scores.append(LABS_score)
        return self.LABS_scores


    def parent_selection(self):
        '''Select two parents with the best LABs score'''
        #print("Selecting parents: \n")
        print("labs scores" + str(self.LABS_scores))
        sorted_indices = np.argsort(self.LABS_scores)[::-1]  # Sort indices in descending order
        print(sorted_indices)
        print("length of LABS scores" + str(len(self.LABS_scores)))
        print("lenght of population" + str(len(self.population)))
        parent1 = self.population[sorted_indices[0]]  # Best individual
        parent2 = self.population[sorted_indices[1]]  # Second best individual

        print("Top two indices:", sorted_indices[:2])

        return parent1, parent2


    def crossover(self, p1, p2):
        if np.random.uniform(0, 1) < self.crossover_rate: 
            crossover_point = np.random.randint(1, self.length)
            part1 = p1[:crossover_point]
            part2 = p2[crossover_point:]
            part3 = p2[:crossover_point]
            part4 = p1[crossover_point:]
        
            child1 = np.concatenate((part1, part2))
            child2 = np.concatenate((part3, part4))


        
            return child1, child2
        else:
            return p1, p2


    def mutate_children(self, child):
        #child = child
        for i in range(len(child)):
            if np.random.rand() < self.mutation_rate:
                child[i] = 1 - child[i]
        
        return child

    def run_mu_lambda(self):
        fitness_progress = []
        print("Generate population: \n")
        self.generate_population()
        print("Generate binary and signed sequences: \n")

        self.generate_binary()
        print("Generating fitness scores for LABS: \n")

        for epoch in range(self.num_epochs):
            
            fitness_scores = self.calculate_LABS()
            #print(fitness_scores)
            p1, p2 = self.parent_selection()
            #print("Parent 1 and 2: \n")
            #print(p1)
            #print(p2)

            best_fitness = max(fitness_scores)
            best_index = fitness_scores.index(best_fitness)
            best_individual = self.population[best_index]
            fitness_progress.append(best_fitness)
            print(f"Epoch {epoch + 1}: Best Fitness = {best_fitness:.4f}")

            sorted_indices = np.argsort(fitness_scores)
            
            new_population = []
            while len(new_population) < self.init_pop_size:
                # Perform crossover
                c1, c2 = self.crossover(p1, p2)

                # Apply mutation
                mutated_c1 = self.mutate_children(c1)
                mutated_c2 = self.mutate_children(c2)

                # Add children to the new population
                new_population.append(mutated_c1)
                if len(new_population) < self.init_pop_size:  # Ensure we don't exceed the size
                    new_population.append(mutated_c2)

            self.population = new_population

            # Early stopping if solution is found
            #if best_fitness == 1.0:
            #   print("Solution found!")
            #    return self.population[sorted_indices[0]], fitness_progress

        print("Best attempt:")
        print("Best individual: ", self.population[sorted_indices[0]])
        return self.population[sorted_indices[0]], fitness_progress



    

        


'''
def run_mu_lambda(self):
        print("Running N-Queens Problem: \n")

       
        fitness_array = []

        for i in range(len(self.population)):
            individual = self.population[i]

            fitness = self.fitness_function(individual)
            fitness_array.append(fitness)


        sort_index = sorted(range(len(fitness_array)), key=lambda x: fitness_array[x], reverse=True)

        best = sort_index[0]
        second_best = sort_index[1]
  
        c1, c2 = self.crossover(np.copy(self.population[best]), np.copy(self.population[second_best]))
        mutated_c1 = self.mutate_children(np.copy(c1))
        mutated_c2 = self.mutate_children(np.copy(c2))

        # Replace the worst individuals with new mutated ones
        worst = sort_index[-1]
        second_worst = sort_index[-2]
        self.population[worst] = mutated_c1
        self.population[second_worst] = mutated_c2

        # Elite: Keep the best individual (optional)
        self.population[0] = np.copy(self.population[best])  # Keep the best individual

        # Early stopping condition if we found the solution
        if self.fitness_function(self.population[best]) == 1:
            print(f"Solution found!")
            return self.population[best]

        return self.population[best]
'''

'\ndef run_mu_lambda(self):\n        print("Running N-Queens Problem: \n")\n\n       \n        fitness_array = []\n\n        for i in range(len(self.population)):\n            individual = self.population[i]\n\n            fitness = self.fitness_function(individual)\n            fitness_array.append(fitness)\n\n\n        sort_index = sorted(range(len(fitness_array)), key=lambda x: fitness_array[x], reverse=True)\n\n        best = sort_index[0]\n        second_best = sort_index[1]\n  \n        c1, c2 = self.crossover(np.copy(self.population[best]), np.copy(self.population[second_best]))\n        mutated_c1 = self.mutate_children(np.copy(c1))\n        mutated_c2 = self.mutate_children(np.copy(c2))\n\n        # Replace the worst individuals with new mutated ones\n        worst = sort_index[-1]\n        second_worst = sort_index[-2]\n        self.population[worst] = mutated_c1\n        self.population[second_worst] = mutated_c2\n\n        # Elite: Keep the best individual (optional)\n     

In [103]:
LABS_instance = LABS()
LABS_instance.run_mu_lambda()

Generate population: 

Generate binary and signed sequences: 

Generating fitness scores for LABS: 

labs scores[4.0, 1.3333333333333333, 4.0, 1.3333333333333333, 1.3333333333333333, 0.5714285714285714, 4.0, 4.0, 1.3333333333333333, 4.0]
[9 7 6 2 0 8 4 3 1 5]
length of LABS scores10
lenght of population10
Top two indices: [9 7]
Epoch 1: Best Fitness = 4.0000
labs scores[4.0, 1.3333333333333333, 4.0, 1.3333333333333333, 1.3333333333333333, 0.5714285714285714, 4.0, 4.0, 1.3333333333333333, 4.0]
[9 7 6 2 0 8 4 3 1 5]
length of LABS scores10
lenght of population10
Top two indices: [9 7]
Epoch 2: Best Fitness = 4.0000
labs scores[4.0, 1.3333333333333333, 4.0, 1.3333333333333333, 1.3333333333333333, 0.5714285714285714, 4.0, 4.0, 1.3333333333333333, 4.0]
[9 7 6 2 0 8 4 3 1 5]
length of LABS scores10
lenght of population10
Top two indices: [9 7]
Epoch 3: Best Fitness = 4.0000
labs scores[4.0, 1.3333333333333333, 4.0, 1.3333333333333333, 1.3333333333333333, 0.5714285714285714, 4.0, 4.0, 1.33333

(array([1, 1, 0, 0]),
 [4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0])

In [40]:
'''def crossover(length, p1, p2, crossover_probability):
    if np.random.uniform(0, 1) < crossover_probability: 
        print("Doing crossover")
        crossover_point = np.random.randint(1, length)
        print("Crossover point" + str(crossover_point))
        part1 = p1[:crossover_point]
        part2 = p2[crossover_point:]
        part3 = p2[:crossover_point]
        part4 = p1[crossover_point:]
        print("part1: " + str(part1))
        print("part2: " + str(part2))
        child1 = np.concatenate((part1, part2))
        child2 = np.concatenate((part3, part4))

        print("Child1 " + str(child1))
        print("Child2 " + str(child2))
    
        return child1, child2
    else:
        return p1, p2
        

crossover_prob = 0.9
p1 = [0, 1, 0, 1]
p2 = [1, 0, 1, 0]
c1,c2 = crossover(4, p1, p2, crossover_prob)

print("c1: " + str(c1))
print("c2: " + str(c2))'''

Doing crossover
Crossover point3
part1: [0, 1, 0]
part2: [0]
Child1 [0 1 0 0]
Child2 [1 0 1 1]
c1: [0 1 0 0]
c2: [1 0 1 1]


In [ ]:
print("Running N-Queens Problem with Mu + Lambda Selection: \n")

        #fitness_array = []
        fitness_progress = []


        for epoch in range(self.num_epochs):
            fitness_scores = [self.fitness_function(ind) for ind in self.population]
            best_fitness = max(fitness_scores)
            fitness_progress.append(best_fitness)

            print(f"Epoch {epoch + 1}: Best Fitness = {best_fitness:.4f}")

            # Sort population by fitness
            sorted_indices = sorted(range(len(fitness_scores)), key=lambda x: fitness_scores[x], reverse=True)
            best, second_best = sorted_indices[:2]

            # Generate offspring
            c1, c2 = self.crossover(self.population[best], self.population[second_best])
            mutated_c1 = self.mutate_children(c1)
            mutated_c2 = self.mutate_children(c2)

            # Append mutated and unmutated offspring to the population
            self.population.extend([mutated_c1, mutated_c2, c1, c2])

            # Limit the population size to μ (initial population size)
            combined_fitness_scores = [self.fitness_function(ind) for ind in self.population]
            sorted_indices = sorted(range(len(combined_fitness_scores)), key=lambda x: combined_fitness_scores[x], reverse=True)
            self.population = [self.population[i] for i in sorted_indices[:self.init_pop_size]]

            if best_fitness == 1.0:
                print("Solution found!")
                return self.population[best], fitness_progress

        print("No solution found. Best attempt:\n")
        print("Best individual: ", self.population[sorted_indices[0]])
        return self.population[sorted_indices[0]], fitness_progress


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9add33c0-a6d3-4fe2-9e64-cc02a9ff9347' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>